<a href="https://colab.research.google.com/github/shubhlohiya/SoS_2019_LeNET/blob/master/LeNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing required packages and libraries.

In [0]:
import torch
import torchvision
import torchvision.transforms as transforms
import torchvision.datasets
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

In [0]:
from bokeh.plotting import figure
from bokeh.io import show, output_notebook
from bokeh.models import LinearAxis, Range1d
import numpy as np

Linking google drive as a data location.

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
DATA_PATH = '/content/gdrive/My Drive/data/MNISTData'
MODEL_STORE_PATH = '/content/gdrive/My Drive/data/'

Initialising Hyperparameters

In [0]:
num_epochs =50
num_classes = 10
batch_size = 250
learning_rate = 0.001

Checking if GPU is available

In [0]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


Setting transform criteria :

In [0]:
trans = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, ), (0.5, ))])


Downloading and loading the train and test sets.

In [0]:
train_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=True, transform=trans, download=True)
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

test_dataset = torchvision.datasets.MNIST(root=DATA_PATH, train=False, transform=trans, download=True)
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

Defining the CNN Architechture

In [0]:
class LeNet(nn.Module):
    def __init__(self):
        super(LeNet, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 6, kernel_size=5, stride=1, padding=2, bias=True),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(6, 16, kernel_size=5, stride=1, padding=0, bias=True),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        self.drop_out = nn.Dropout()
        self.fc1 = nn.Linear(16*5*5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = out.view(-1, 16*5*5)
        out = self.drop_out(out)
        out = F.relu(self.fc1(out))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out

Creating an instance of the CNN

In [0]:
model = LeNet()
medel = model.to(device)

Setting the Loss Function and optimization criteria

In [0]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

Training the CNN on the train dataset

In [0]:
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        # Run the forward pass
        images=images.to(device)
        labels=labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())

        # Backprop and perform Adam optimisation
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Track the accuracy
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)

        if (i + 1) % 120 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))


Epoch [1/50], Step [120/240], Loss: 0.3756, Accuracy: 87.60%
Epoch [1/50], Step [240/240], Loss: 0.3322, Accuracy: 90.80%
Epoch [2/50], Step [120/240], Loss: 0.1508, Accuracy: 95.60%
Epoch [2/50], Step [240/240], Loss: 0.1255, Accuracy: 96.40%
Epoch [3/50], Step [120/240], Loss: 0.1083, Accuracy: 96.80%
Epoch [3/50], Step [240/240], Loss: 0.1144, Accuracy: 96.40%
Epoch [4/50], Step [120/240], Loss: 0.1010, Accuracy: 95.20%
Epoch [4/50], Step [240/240], Loss: 0.1377, Accuracy: 95.60%
Epoch [5/50], Step [120/240], Loss: 0.0848, Accuracy: 97.20%
Epoch [5/50], Step [240/240], Loss: 0.0764, Accuracy: 97.20%
Epoch [6/50], Step [120/240], Loss: 0.0331, Accuracy: 98.80%
Epoch [6/50], Step [240/240], Loss: 0.1073, Accuracy: 96.80%
Epoch [7/50], Step [120/240], Loss: 0.0666, Accuracy: 97.60%
Epoch [7/50], Step [240/240], Loss: 0.0597, Accuracy: 96.80%
Epoch [8/50], Step [120/240], Loss: 0.0849, Accuracy: 96.40%
Epoch [8/50], Step [240/240], Loss: 0.0759, Accuracy: 97.20%
Epoch [9/50], Step [120/

Testing the classifier on the test set

In [0]:
model.eval()
with torch.no_grad():
    correct = 0
    total = 0
    i=0
    for images, labels in test_loader:
        images=images.to(device)
        labels=labels.to(device)
        outputs = model(images)
        i+=1
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    print('Test Accuracy of the model on the {} test images: {} %'.format(i*batch_size,(correct / total) * 100))

# Save the model and plot
torch.save(model.state_dict(), MODEL_STORE_PATH + 'conv_net_model.ckpt')
output_notebook()
p = figure(y_axis_label='Loss', width=850, y_range=(0, 1), title='LeNet results')
p.extra_y_ranges = {'Accuracy': Range1d(start=0, end=100)}
p.add_layout(LinearAxis(y_range_name='Accuracy', axis_label='Accuracy (%)'), 'right')
p.line(np.arange(len(loss_list)), loss_list)
p.line(np.arange(len(loss_list)), np.array(acc_list) * 100, y_range_name='Accuracy', color='red')
show(p)

Test Accuracy of the model on the 10000 test images: 99.26 %


Loading BokehJS ...